In [1]:

#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Helper libraries
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
import math

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, Xception, MobileNet, DenseNet121, NASNetMobile, EfficientNetB0, MobileNetV2, MobileNetV3Large
from tensorflow.python.keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D
from keras.models import load_model
from keras.utils.vis_utils import plot_model


2023-03-27 21:40:53.753236: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 21:40:53.998100: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-27 21:40:55.163360: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/
2023-03-27 21:40:55.163482: W t

Num GPUs Available:  1


In [2]:
batch_size = 16
img_size = [240, 320]

# df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
# df['filename'] = df["image_id"].astype(str) + ".png"
# df = df.sample(frac=1).reset_index(drop=True)
# df.to_csv("training_norm_shuffle.csv", encoding='utf-8', index=False)

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm_shuffle.csv')
print(df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


       image_id   angle  speed   filename
0           414  0.8125    1.0    414.png
1         17532  0.6875    1.0  17532.png
2         16480  0.3125    0.0  16480.png
3          6450  0.6875    0.0   6450.png
4         12314  0.6250    0.0  12314.png
...         ...     ...    ...        ...
14821      9844  0.6875    1.0   9844.png
14822     12367  0.5000    1.0  12367.png
14823       343  0.9375    1.0    343.png
14824      3838  0.6875    0.0   3838.png
14825     13301  0.5000    1.0  13301.png

[14826 rows x 4 columns]


In [3]:

shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    # width_shift_range=shift,
    # height_shift_range=shift
)

speed_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["speed"],
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

angle_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle"],
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

speed_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

angle_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

speed_eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

angle_eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')



Found 8895 validated image filenames.
Found 8895 validated image filenames.
Found 2965 validated image filenames.
Found 2965 validated image filenames.
Found 2966 validated image filenames.
Found 2966 validated image filenames.


In [4]:
filename = os.listdir("machine-learning-in-science-ii-2023/test_data/test_data") 

df = pd.DataFrame(filename)
df.columns = ["filename"]

df[['file', 'type']] = df.filename.str.split(".", expand = True)
df["file"] = df["file"].astype(str).astype(int)

df.sort_values(by=['file'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)
test_datagen = ImageDataGenerator(rescale = 1./255)

test_images = test_datagen.flow_from_dataframe(
    dataframe=df,
    directory="machine-learning-in-science-ii-2023/test_data/test_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

      filename  file type
0        1.png     1  png
1        2.png     2  png
2        3.png     3  png
3        4.png     4  png
4        5.png     5  png
...        ...   ...  ...
1015  1016.png  1016  png
1016  1017.png  1017  png
1017  1018.png  1018  png
1018  1019.png  1019  png
1019  1020.png  1020  png

[1020 rows x 3 columns]
Found 1020 validated image filenames.


In [5]:
shape = (*img_size, 3) # inherited image size with 3 color filters
input_shape = [240, 320, 3]
transfer = MobileNetV2(
    input_shape=input_shape, 
    include_top=False, 
    weights='imagenet',
    classifier_activation="relu"
)

for layer in transfer.layers[:-2]:
    layer.trainable = False

2023-03-27 21:40:57.408694: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 21:40:57.411913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-27 21:40:57.411982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-27 21:40:57.412003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0

In [6]:
# Speed CNN base
speed_model = models.Sequential()

speed_model.add(transfer)
speed_model.build()
        
speed_model.add(layers.Conv2D(128, (3, 3), strides=(2,2), activation='relu'))
speed_model.add(layers.BatchNormalization())

speed_model.add(layers.Flatten()),

speed_model.add(layers.Dense(128, activation='relu'))
speed_model.add(layers.BatchNormalization())
speed_model.add(layers.Dropout(0.2))

speed_model.add(layers.Dense(64, activation='relu'))
speed_model.add(layers.BatchNormalization())
speed_model.add(layers.Dropout(0.2))

# Output layer
speed_model.add(layers.Flatten())
speed_model.add(layers.Dense(1,  activation='sigmoid', kernel_initializer='normal'))

In [7]:
# Angle CNN base
angle_model = models.Sequential()

angle_model.add(transfer)
angle_model.build()
        
angle_model.add(layers.Conv2D(128, (3, 3), strides=(2,2), activation='relu'))
angle_model.add(layers.BatchNormalization())

angle_model.add(layers.Flatten()),

# model_1.add(layers.Dense(128, activation='relu'))
angle_model.add(layers.Dense(128, activation='relu'))
angle_model.add(layers.BatchNormalization())
angle_model.add(layers.Dropout(0.2))

# Output layer
angle_model.add(layers.Flatten())
angle_model.add(layers.Dense(1,  activation='relu', kernel_initializer='normal'))

In [8]:
#speed_model.build()
angle_model.build()

In [9]:
#speed_model.summary()
#angle_model.summary()

In [10]:
speed_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

angle_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

In [11]:
# history = speed_model.fit(
#     speed_train_generator,
#     batch_size=batch_size,
#     validation_data=speed_val_generator,
#     callbacks=[es],
#     epochs=100
# )

In [12]:
# test_loss = speed_model.evaluate(
#     speed_eval_generator,
#     verbose=1
# )

In [13]:
history = angle_model.fit(
    angle_train_generator,
    batch_size=batch_size,
    validation_data=angle_val_generator,
    callbacks=[es],
    epochs=100
)

Epoch 1/100


2023-03-27 21:41:12.362615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-27 21:41:16.073939: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2023-03-27 21:41:18.199377: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-27 21:41:18.232764: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7ff6e8b375b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-27 21:41:18.232808: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2023-03-27 21:41:18.268907: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-27 21:41:18.53

556/556 [==============================] - 51s 60ms/step - loss: 0.0840 - val_loss: 0.0200
Epoch 2/100
556/556 [==============================] - 30s 54ms/step - loss: 0.0164 - val_loss: 0.0099
Epoch 3/100
556/556 [==============================] - 27s 49ms/step - loss: 0.0107 - val_loss: 0.0096
Epoch 4/100
556/556 [==============================] - 26s 47ms/step - loss: 0.0092 - val_loss: 0.0086
Epoch 5/100
556/556 [==============================] - 26s 47ms/step - loss: 0.0079 - val_loss: 0.0087
Epoch 6/100
556/556 [==============================] - 26s 47ms/step - loss: 0.0074 - val_loss: 0.0083
Epoch 7/100
556/556 [==============================] - 26s 47ms/step - loss: 0.0066 - val_loss: 0.0076
Epoch 8/100
556/556 [==============================] - 26s 47ms/step - loss: 0.0059 - val_loss: 0.0092
Epoch 9/100
556/556 [==============================] - 26s 47ms/step - loss: 0.0055 - val_loss: 0.0103
Epoch 10/100
556/556 [==============================] - 27s 49ms/step - loss: 0.0050 

In [14]:
test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)

93/93 [==============================] - 8s 83ms/step - loss: 0.0066


In [15]:
angle_prediction = angle_model.predict(test_images)
speed_prediction = speed_model.predict(test_images)

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

submission = {"angle":angle_prediction , "speed":speed_prediction}

df = pd.DataFrame(submission)
df.index += 1 
df.to_csv('Submission.csv')


32/32 [==============================] - 3s 67ms/step


ValueError: Per-column arrays must each be 1-dimensional

In [ ]:
diff = abs(x_evaluate["angle"] - a)
results = pd.DataFrame({"Train":x_evaluate["angle"], "Predicted":a, "Difference":diff})
#results

In [ ]:
results = pd.DataFrame({"Train":x_evaluate["speed"],"Predicted":b})
#results

In [ ]:
speed_model.save("speed-model.h5")
angle_model.save("angle-model.h5")

In [ ]:
model2 = load_model('model.hdf5')

transfer = model2.layers[0]
transfer.trainable = True


for layer in transfer.layers[:-25]:
    layer.trainable = False

#model2.trainable = True

# for layer in model2.layers[:2]:
#     layer.trainable = False
    
model2.summary()


In [ ]:

model2.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001), # 1e-5 
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0005,
    patience=30,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model2.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=300
)


In [ ]:
from sklearn.metrics import classification_report

test_loss = model2.evaluate(
    eval_generator,
    verbose=1
)


In [ ]:
model2.save("model-fine-tuning.hdf5")

In [ ]:

prediction = model2.predict(test_images)
print(prediction.shape)
import math

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission-fine-tuning.csv')


In [ ]:
model = load_model("model-fine-tuning.hdf5")
plot_model(model, show_layer_names=True, show_shapes=True,show_layer_activations=True)